In [ ]:
import os
from IPython.display import Image
import openai
from azure.storage.blob import BlobServiceClient

import utils
from dotenv import load_dotenv
load_dotenv(".env")

### SET VARIABLES

In [ ]:
aoai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
aoai_api_key = os.environ["AZURE_OPENAI_API_KEY"]
aoai_api_version = os.environ["AZURE_OPENAI_API_VERSION"]
aoai_embedding_model = os.environ["AZURE_OPENAI_EMBEDDING_MODEL"]
aoai_chat_model = os.environ["AZURE_OPENAI_CHAT_MODEL"]

client = openai.AzureOpenAI(
    azure_endpoint=aoai_endpoint,
    api_key=aoai_api_key,
    api_version=aoai_api_version
)
    
connection_string = os.environ["STORAGE_CONN_STR"]
storage_sas_token = os.environ["STORAGE_SAS_TOKEN"]
container_name = os.environ["CONTAINER_NAME"] 
separator_word = os.environ["SEPARATOR_WORD"]

### PERFORM A HYBRID SEARCH

In [ ]:
import json
query = "ペットボトルの投棄方法は 1 から 9 番のどれですか？" 
answer_context = utils.search_index(query, client, aoai_embedding_model)
answer_context

### RAG Query to gpt-4o

In [ ]:
with open("./system_message_01.txt", "r", encoding = 'utf-8') as f:
    system_message = f.read()

In [ ]:
messages = []
messages.append({"role": "system","content": system_message})

content = {}
content["question"] = query
content["context"] = answer_context
messages.append({"role": "user", "content": str(content)})


In [ ]:
response = utils.gpt4o_query(messages, client, aoai_chat_model)
answer = response.choices[0].message.content

answer = json.loads(answer)
print(answer)

In [ ]:
for num, item in enumerate(answer["answers"]):
    if not item["answer"].startswith("Sorry"):
        print("answer["+ str(num) + "]: ", item["answer"])
        print("  page["+ str(num) + "]: ", item["page"])

### Get URL of the 'title' image

In [ ]:
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
image_urls = []

for item in answer["answers"]:
    if not item["answer"].startswith("Sorry"):
        blob_name = item["page"]
        image_titles_and_urls = utils.list_blobs_titles_and_urls(blob_service_client, container_name, blob_name)
        if image_titles_and_urls:
            image_urls.append(image_titles_and_urls)

if len(image_urls) == 0:
    print("No images found.")

image_urls

## Sending a new message to gpt-4o vision

### Construct a new message with a single image

In [ ]:
with open("./system_message_02.txt", "r", encoding = 'utf-8') as f:
    system_message = f.read()

In [ ]:
answer_string = ', '.join(json.dumps(item, ensure_ascii=False) for item in answer["answers"] \
                          if not item["answer"].startswith("Sorry"))
answer_string

In [ ]:
messages = []
messages.append({"role": "system","content": system_message})

content = []
content.append({"type": "text", "text": "query: " + query})
content.append({"type": "text", "text":"answer from gpt-4o: " + answer_string})

for url in image_urls:    # pick up the relevant images
    storagepath = url['title']
    storagepath_stem = os.path.splitext(storagepath)[0]
    if not (storagepath_stem[-3] == separator_word and storagepath_stem[-2:].isdigit()):
        storagepath_stem += separator_word + '01' # storagepath doesn't have a number suffix
    for item in answer["answers"]:
        if storagepath_stem == item['page']:
            content.append({"type": "text", "text": storagepath_stem})
            content.append({"type": "image_url", "image_url": {"url": url['url']+storage_sas_token}})
            break

messages.append({"role": "user","content":content})

In [ ]:
messages

### Ask gpt-4o

In [ ]:
response = utils.gpt4o_query(messages, client, aoai_chat_model)
answer = response.choices[0].message.content

answer = json.loads(answer)
print(answer)

In [ ]:
print(answer["answer"])
print(answer["citation"])

### Download and show the image

In [ ]:
for url in image_urls:    # pick up the relevant images
    storagepath = url['title']

    utils.list_blobs_download(blob_service_client, container_name, storagepath)
    print("\n",storagepath)
    display(Image(filename=storagepath))

## Maintain the massage

In [ ]:
utils.append_conversation_history(messages, response, role="assistant")
print(messages)

### Send the consecutive query

In [ ]:
messages.append({"role": "user","content":[
    {"type": "text", "text": "ビンは何番ですか？"}, 
]})
response = utils.gpt4o_query(messages, client, aoai_chat_model)
answer = response.choices[0].message.content

answer = json.loads(answer)
print(answer)

### END OF SCRIPT